# Census Notebook
**Authorship**<br />
Original Author: Taurean Dyer<br />
Last Edit: Taurean Dyer, 9/26/2019<br />

**Test System Specs**<br />
Test System Hardware: GV100<br />
Test System Software: Ubuntu 18.04<br />
RAPIDS Version: 0.10.0a - Docker Install<br />
Driver: 410.79<br />
CUDA: 10.0<br />


**Known Working Systems**<br />
RAPIDS Versions:0.8, 0.9, 0.10

# Intro
Held every 10 years, the US census gives a detailed snapshot in time about the makeup of the country.  The last census in 2010 surveyed nearly 309 million people.  IPUMS.org provides researchers an open source data set with 1% to 10% of the census data set.  In this notebook, we want to see how education affects total income earned in the US based on data from each census from the 1970 to 2010 and see if we can predict some results if the census was held today, according to the national average.  We will go through the ETL, training the model, and then testing the prediction.  We'll make every effort to get as balanced of a dataset as we can.  We'll also pull some extra variables to allow for further self-exploration of gender based education and income breakdowns.  On a single Titan RTX, you can run the whole notebook workflow on the 4GB dataset of 14 million rows by 44 columns in less than 3 minutes.  

**Let's begin!**

## Imports

In [1]:
import pandas as pd
import numpy as np
import cuml
import cudf
import dask_cudf
import sys
import os
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')

## Get your data!

In [2]:
import urllib.request
import time

from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster

cluster = LocalCUDACluster()
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:33355 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 16.48 GB


The ipums dataset is in our S3 bucket and zipped.  
1. We'll need to create a folder for our data in the `/data` folder
1. Download the zipped data into that folder from S3
1. Load the zipped data quickly into cudf using it's read_csv() parameters

In [3]:
data_dir = '../data/census/'
if not os.path.exists(data_dir):
    print('creating census data directory')
    os.system('mkdir ../data/census/')

creating census data directory


In [5]:
# download the IPUMS dataset
base_url = 'https://rapidsai-data.s3.us-east-2.amazonaws.com/datasets/'
fn = 'ipums_education2income_1970-2010.csv.gz'
if not os.path.isfile(data_dir+fn):
    print(f'Downloading {base_url+fn} to {data_dir+fn}')
    urllib.request.urlretrieve(base_url+fn, data_dir+fn)

In [6]:
def load_data(cached = data_dir+fn):
    if os.path.exists(cached):
        print('use ipums data')
        X = cudf.read_csv(cached, compression='infer')
    else:
        print("No data found!  Please check your that your data directory is ../../data/census/ and that you downloaded the data.  If you did, please delete the `../../../data/census/` directory and try the above 2 cells again")
        X = null
    return X

In [7]:
df = load_data(data_dir+fn)
# limit
df = df[0:100]
print('data',df.shape)


use ipums data
data (100, 45)


In [8]:
print(df.head(5).to_pandas())

   YEAR  DATANUM  SERIAL  CBSERIAL  HHWT  CPI99  GQ  QGQ  PERNUM  PERWT  ...  \
0  1970        2       1       NaN   100   4.54   1  0.0       1    100  ...   
1  1970        2       1       NaN   100   4.54   1  0.0       2    100  ...   
2  1970        2       2       NaN   100   4.54   1  0.0       1    100  ...   
3  1970        2       2       NaN   100   4.54   1  0.0       2    100  ...   
4  1970        2       4       NaN   100   4.54   1  0.0       1    100  ...   

   EDUCD_POP  EDUCD_SP  EDUCD_MOM2  EDUCD_POP2  INCTOT_HEAD  INCTOT_MOM  \
0        NaN      30.0         NaN         NaN      12450.0         NaN   
1        NaN      60.0         NaN         NaN      12450.0         NaN   
2        NaN      60.0         NaN         NaN       9050.0         NaN   
3        NaN      70.0         NaN         NaN       9050.0         NaN   
4        NaN      23.0         NaN         NaN       7450.0         NaN   

   INCTOT_POP  INCTOT_SP  INCTOT_MOM2  INCTOT_POP2  
0         NaN  

In [9]:
df.dtypes

YEAR             int64
DATANUM          int64
SERIAL           int64
CBSERIAL       float64
HHWT             int64
CPI99          float64
GQ               int64
QGQ            float64
PERNUM           int64
PERWT            int64
SEX              int64
AGE              int64
EDUC             int64
EDUCD            int64
INCTOT           int64
SEX_HEAD       float64
SEX_MOM        float64
SEX_POP        float64
SEX_SP         float64
SEX_MOM2       float64
SEX_POP2       float64
AGE_HEAD       float64
AGE_MOM        float64
AGE_POP        float64
AGE_SP         float64
AGE_MOM2       float64
AGE_POP2       float64
EDUC_HEAD      float64
EDUC_MOM       float64
EDUC_POP       float64
EDUC_SP        float64
EDUC_MOM2      float64
EDUC_POP2      float64
EDUCD_HEAD     float64
EDUCD_MOM      float64
EDUCD_POP      float64
EDUCD_SP       float64
EDUCD_MOM2     float64
EDUCD_POP2     float64
INCTOT_HEAD    float64
INCTOT_MOM     float64
INCTOT_POP     float64
INCTOT_SP      float64
INCTOT_MOM2

In [10]:
original_counts = df.YEAR.value_counts()
print(original_counts) ### Remember these numbers!

1970    100
Name: YEAR, dtype: int32


## ETL

### Cleaning Income data
First, let's focus on cleaning out the bad values for Total Income `INCTOT`. First, let's see if there are an `N/A` values, as when we did `head()`, we saw some in other columns, like CBSERIAL

In [11]:
df['INCTOT_NA'] = df['INCTOT'].isna()

In [12]:
print(df.INCTOT_NA.value_counts())

False    100
Name: INCTOT_NA, dtype: int32


Okay, great, there are no `N/A`s...or are there?  Let's drop `INCTOT_NA` and see what our value counts look like

In [13]:
df=df.drop('INCTOT_NA', axis=1)
print(df.INCTOT.value_counts().to_pandas())  ### Wow, look how many people in America make $10,000,000!  Wait a minutes... 

9999999    21
0          14
250         4
4050        3
5050        3
350         3
2050        3
9050        2
4850        2
650         2
2150        2
1250        2
22150       1
5650        1
17850       1
1050        1
11250       1
5550        1
50          1
16850       1
11150       1
4350        1
7450        1
13350       1
6950        1
25050       1
1850        1
2450        1
17150       1
6150        1
12450       1
11450       1
4550        1
50000       1
550         1
8850        1
8050        1
6050        1
19350       1
2950        1
150         1
1150        1
2750        1
7150        1
15050       1
7750        1
3450        1
5350        1
7050        1
950         1
12050       1
Name: INCTOT, dtype: int32


Not that many people make $10M a year. Checking https://usa.ipums.org/usa-action/variables/INCTOT#codes_section, `9999999`is INCTOT's code for `N/A`.  That was why when we ran `isna`, RAPIDS won't find any.  Let's first create a new dataframe that is only NA values, then let's pull those encoded `N/A`s out of our working dataframe!

In [14]:
print('data',df.shape)
tdf = df.query('INCTOT == 9999999')
df = df.query('INCTOT != 9999999')

data (100, 45)


In [15]:
print('working data',df.shape)
print('junk count data',tdf.shape)

working data (79, 45)
junk count data (21, 45)


We're down by nearly 1/4 of our original dataset size.  For the curious, now we should be able to get accurate Total Income data, by year, not taking into account inflation

In [16]:
print(df.groupby('YEAR')['INCTOT'].mean()) # without that cleanup, the average would have bene in the millions....

YEAR
1970    5503.797468
Name: INCTOT, dtype: float64


#### Normalize Income for inflation
Now that we have reduced our dataframe to a baseline clean data to answer our question, we should normalize the amounts for inflation.  `CPI99`is the value that IPUMS uses to contian the inflation factor.  All we have to do is multipy by year.  Let's see how that changes the Total Income values from just above!

In [17]:
print(df.groupby('YEAR')['CPI99'].mean()) ## it just returns the CPI99
df['INCTOT'] = df['INCTOT'] * df['CPI99']
print(df.groupby('YEAR')['INCTOT'].mean()) ## let's see what we got!

YEAR
1970    4.54
Name: CPI99, dtype: float64
YEAR
1970    24987.240506
Name: INCTOT, dtype: float64


### Cleaning Education Data
Okay, great!  Now we have income cleaned up, it should also have cleaned much of our next sets of values of interes, namely Education and Education Detailed.  However, there are still some `N/A`s in key variables to worry about, which can cause problmes later.  Let's create a list of them...

In [18]:
suspect = ['CBSERIAL','EDUC', 'EDUCD', 'EDUC_HEAD', 'EDUC_POP', 'EDUC_MOM','EDUCD_MOM2','EDUCD_POP2', 'INCTOT_MOM','INCTOT_POP','INCTOT_MOM2','INCTOT_POP2', 'INCTOT_HEAD']

In [19]:
for i in range(0, len(suspect)):
    df[suspect[i]] = df[suspect[i]].fillna(-1)
    print(suspect[i], df[suspect[i]].value_counts())

CBSERIAL -1.0    79
Name: CBSERIAL, dtype: int32
EDUC 6     22
2     15
10     8
3      7
4      7
11     6
8      5
7      4
1      3
5      1
9      1
Name: EDUC, dtype: int32
EDUCD 60     20
26      9
100     8
30      7
40      7
111     6
80      5
70      4
25      3
23      2
14      2
65      2
90      1
50      1
22      1
17      1
Name: EDUCD, dtype: int32
EDUC_HEAD  6.0     13
 11.0    12
 8.0     10
 2.0     10
 7.0      9
 10.0     9
 4.0      6
 1.0      4
 3.0      3
 5.0      2
-1.0      1
Name: EDUC_HEAD, dtype: int32
EDUC_POP -1.0     68
 8.0      3
 7.0      3
 2.0      2
 11.0     2
 10.0     1
Name: EDUC_POP, dtype: int32
EDUC_MOM -1.0     64
 6.0      8
 10.0     3
 3.0      2
 7.0      1
 2.0      1
Name: EDUC_MOM, dtype: int32
EDUCD_MOM2 -1.0    79
Name: EDUCD_MOM2, dtype: int32
EDUCD_POP2 -1.0    79
Name: EDUCD_POP2, dtype: int32
INCTOT_MOM -1.0       64
 0.0        4
 650.0      3
 2050.0     2
 4850.0     2
 1150.0     2
 7750.0     1
 1250.0     1
Name: INC

Let's get drop any rows of any `-1`s in Education and Education Detailed.

In [20]:
totincome = ['EDUC','EDUCD']
for i in range(0, len(totincome)):
    query = totincome[i] + ' != -1'
    df = df.query(query)
    print(totincome[i])

EDUC
EDUCD


In [21]:
print(df.shape)
df.head().to_pandas().head()

(79, 45)


,YEAR,DATANUM,SERIAL,CBSERIAL,HHWT,CPI99,GQ,QGQ,PERNUM,PERWT,...,EDUCD_POP,EDUCD_SP,EDUCD_MOM2,EDUCD_POP2,INCTOT_HEAD,INCTOT_MOM,INCTOT_POP,INCTOT_SP,INCTOT_MOM2,INCTOT_POP2
0,1970,2,1,-1.0,100,4.54,1,0.0,1,100,...,NaN,30.0,-1.0,-1.0,12450.0,-1.0,-1.0,3450.0,-1.0,-1.0
1,1970,2,1,-1.0,100,4.54,1,0.0,2,100,...,NaN,60.0,-1.0,-1.0,12450.0,-1.0,-1.0,12450.0,-1.0,-1.0
2,1970,2,2,-1.0,100,4.54,1,0.0,1,100,...,NaN,60.0,-1.0,-1.0,9050.0,-1.0,-1.0,0.0,-1.0,-1.0
3,1970,2,2,-1.0,100,4.54,1,0.0,2,100,...,NaN,70.0,-1.0,-1.0,9050.0,-1.0,-1.0,9050.0,-1.0,-1.0
4,1970,2,4,-1.0,100,4.54,1,0.0,1,100,...,NaN,23.0,-1.0,-1.0,7450.0,-1.0,-1.0,650.0,-1.0,-1.0


Well, the good news is that we lost no further rows, start to normalize the data so when we do our OLS, one year doesn't unfairly dominate the data

## Normalize the Data
The in the last step, need to keep our data at about the same ratio as we when started (1% of the population), with the exception of 1980, which was a 5% and needs to be reduced.  This is why we kept the temp dataframe `tdf` - to get the counts per year.   we will find out just how many have to realize

In [22]:
print('Working data: \n', df.YEAR.value_counts())
print('junk count data: \n', tdf.YEAR.value_counts())

Working data: 
 1970    79
Name: YEAR, dtype: int32
junk count data: 
 1970    21
Name: YEAR, dtype: int32


And now, so that we can do MSE, let's make all the dtypes the same. 

In [23]:
df.dtypes

YEAR             int64
DATANUM          int64
SERIAL           int64
CBSERIAL       float64
HHWT             int64
CPI99          float64
GQ               int64
QGQ            float64
PERNUM           int64
PERWT            int64
SEX              int64
AGE              int64
EDUC             int64
EDUCD            int64
INCTOT         float64
SEX_HEAD       float64
SEX_MOM        float64
SEX_POP        float64
SEX_SP         float64
SEX_MOM2       float64
SEX_POP2       float64
AGE_HEAD       float64
AGE_MOM        float64
AGE_POP        float64
AGE_SP         float64
AGE_MOM2       float64
AGE_POP2       float64
EDUC_HEAD      float64
EDUC_MOM       float64
EDUC_POP       float64
EDUC_SP        float64
EDUC_MOM2      float64
EDUC_POP2      float64
EDUCD_HEAD     float64
EDUCD_MOM      float64
EDUCD_POP      float64
EDUCD_SP       float64
EDUCD_MOM2     float64
EDUCD_POP2     float64
INCTOT_HEAD    float64
INCTOT_MOM     float64
INCTOT_POP     float64
INCTOT_SP      float64
INCTOT_MOM2

In [24]:

keep_cols = ['YEAR', 'DATANUM', 'SERIAL', 'CBSERIAL', 'HHWT', 'GQ', 'PERNUM', 'SEX', 'AGE', 'INCTOT', 'EDUC', 'EDUCD', 'EDUC_HEAD', 'EDUC_POP', 'EDUC_MOM','EDUCD_MOM2','EDUCD_POP2', 'INCTOT_MOM','INCTOT_POP','INCTOT_MOM2','INCTOT_POP2', 'INCTOT_HEAD', 'SEX_HEAD']
df = df.loc[:, keep_cols]
#df = df.drop(col for col in df.columns if col not in keep_cols)
for i in range(0, len(keep_cols)):
    df[keep_cols[i]] = df[keep_cols[i]].fillna(-1)
    print(keep_cols[i], df[keep_cols[i]].value_counts())
    df[keep_cols[i]]= df[keep_cols[i]].astype('float64')

YEAR 1970    79
Name: YEAR, dtype: int32
DATANUM 2    79
Name: DATANUM, dtype: int32
SERIAL 18    5
14    4
37    4
31    4
32    3
39    3
16    3
10    3
5     2
26    2
25    2
20    2
38    2
13    2
21    2
4     2
36    2
1     2
19    2
33    2
22    2
7     2
17    2
9     2
27    2
2     2
15    2
34    2
29    2
23    1
28    1
6     1
30    1
24    1
35    1
11    1
8     1
Name: SERIAL, dtype: int32
CBSERIAL -1.0    79
Name: CBSERIAL, dtype: int32
HHWT 100    79
Name: HHWT, dtype: int32
GQ 1    78
3     1
Name: GQ, dtype: int32
PERNUM 1    37
2    29
3     8
4     4
5     1
Name: PERNUM, dtype: int32
SEX 2    42
1    37
Name: SEX, dtype: int32
AGE 32    4
54    4
14    3
40    3
36    3
31    3
15    3
23    2
64    2
61    2
25    2
20    2
77    2
66    2
62    2
35    2
47    2
43    2
41    2
55    2
16    2
56    1
38    1
52    1
63    1
65    1
44    1
86    1
30    1
59    1
21    1
75    1
49    1
37    1
19    1
39    1
98    1
79    1
70    1
18    1
22    1
17  

In [25]:
## I WANTED TO REDUCE THE 1980 SAMPLE HERE, BUT .SAMPLE() IS NEEDED AND NOT WORKING, UNLESS THERE IS A WORK AROUND...

With the important data now clean and normalized, let's start doing the regression

## Ridge Regression
We have 44 variables.  The other variables may provide important predictive information.  The Ridge Regression technique with cross validation to identify the best hyperparamters may be the best way to get the most accurate model.  We'll have to 

* define our performance metrics
* split our data into train and test sets
* train and test our model

Let's begin and see what we get!

In [26]:
# As our performance metrics we'll use a basic mean squared error and coefficient of determination implementation
def mse(y_test, y_pred):
    return ((y_test.reset_index(drop=True) - y_pred.reset_index(drop=True)) ** 2).mean()

def cod(y_test, y_pred):
    y_bar = y_test.mean()
    total = ((y_test - y_bar) ** 2).sum()
    residuals = ((y_test.reset_index(drop=True) - y_pred.reset_index(drop=True)) ** 2).sum()
    return 1 - (residuals / total)

In [27]:
from cuml.preprocessing.model_selection import train_test_split
trainsize = .9
yCol = "EDUC"
from cuml.preprocessing.model_selection import train_test_split
from cuml.linear_model.ridge import Ridge

def train_and_score(data, clf, train_frac=0.8, n_runs=20):
    mse_scores, cod_scores = [], []
    for _ in range(n_runs):
        X_train, X_test, y_train, y_test = cuml.preprocessing.model_selection.train_test_split(df, yCol, train_size=.9)
        y_pred = clf.fit(X_train, y_train).predict(X_test)
        mse_scores.append(mse(y_test, y_pred))
        cod_scores.append(cod(y_test, y_pred))
    return mse_scores, cod_scores

 ## Results
 **Moment of truth!  Let's see how our regression training does!**

In [28]:
import numpy as np
n_runs = 20
clf = Ridge()
mse_scores, cod_scores = train_and_score(df, clf, n_runs=n_runs)
print(f"median MSE ({n_runs} runs): {np.median(mse_scores)}")
print(f"median COD ({n_runs} runs): {np.median(cod_scores)}")

median MSE (20 runs): 0.0373372816561586
median COD (20 runs): 0.9951869736898369


**Fun fact:** if you made INCTOT the y axis, your prediction results would not be so pretty!  It just shows that your education level can be an indicator for your income, but your income is NOT a great predictor for your education level.  You have better odds flipping a coin!

* median MSE (50 runs): 518189521.07548225
* median COD (50 runs): 0.425769113846303

## Next Steps/Self Study
* You can pickle the model and use it in another workflow
* You can redo the workflow with based on head of household using `EDUC`, `SEX`, and `INCTOT` for X in `X`_HEAD
* You can see the growing role of education with women in their changing role in the workforce and income with "EDUC_MOM" and "EDUC_POP